In [1]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("labels_final.csv")
data.shape
labels = data["label"].unique()
print("The number of categories here are:", len(labels))
print("The labels are:", labels)

The number of categories here are: 16
The labels are: [ 3  2  7  4  1 13  5  6  0 15 11  8 14 10 12  9]


In [3]:
#ref: https://vijayabhaskar96.medium.com/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255, validation_split = 0.3)
train_data = datagen.flow_from_dataframe(dataframe = data, directory = "data_final", x_col = "path", y_col = "label", subset = "training", batch_size = 32, seed = 42, shuffle = True, class_mode = "raw", target_size = (224,224))
valid_data = datagen.flow_from_dataframe(dataframe = data, directory = "data_final", x_col = "path", y_col = "label", subset = "validation", batch_size = 32, seed = 42, shuffle = True, class_mode = "raw", target_size = (224,224))


Found 33600 validated image filenames.
Found 14400 validated image filenames.


In [4]:


image_size = [224,224]
from tensorflow.keras.layers import Dense,concatenate,Activation,Dropout,Input,LSTM, MaxPooling2D,Conv2D,Embedding,Flatten
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model

tf.keras.backend.clear_session()
vgg1 = tf.keras.applications.VGG16(include_top = False, weights = 'imagenet', input_shape = image_size + [3])
for layers in vgg1.layers:
    layers.trainable = False   
layer1 = Conv2D(64,(3,3))(vgg1.output)
layer2 = MaxPooling2D()(layer1)
layer3 = Dense(64, activation ='relu')(layer2)
layer4 = Dense(64, activation = 'relu')(layer3)
flatten = Flatten()(layer4)
Output = Dense(16, activation = 'softmax')(flatten)

model = Model(inputs = vgg1.input, outputs = Output)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
#ref:#ref: https://vijayabhaskar96.medium.com/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
tb_model = tf.keras.callbacks.TensorBoard(log_dir = "logs_model/", histogram_freq = 0)
model.compile(optimizer = optimizer, loss= tf.keras.losses.SparseCategoricalCrossentropy(), metrics = ['accuracy'])
step_size_train = train_data.n//train_data.batch_size
step_size_val = valid_data.n//valid_data.batch_size
history1 = model.fit_generator(generator = train_data, steps_per_epoch = step_size_train, validation_data = valid_data, validation_steps = step_size_val, epochs = 5, callbacks=[tb_model])

Epoch 1/5


C:\Users\THISPC~1\AppData\Local\Temp/ipykernel_11952/467474330.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history1 = model.fit_generator(generator = train_data, steps_per_epoch = step_size_train, validation_data = valid_data, validation_steps = step_size_val, epochs = 5, callbacks=[tb_model])


1050/1050 [==============================] - 298s 283ms/step - loss: 1.4318 - accuracy: 0.5614 - val_loss: 1.1688 - val_accuracy: 0.6524
Epoch 2/5
1050/1050 [==============================] - 300s 285ms/step - loss: 1.0697 - accuracy: 0.6772 - val_loss: 1.1395 - val_accuracy: 0.6611
Epoch 3/5
1050/1050 [==============================] - 300s 285ms/step - loss: 0.9201 - accuracy: 0.7208 - val_loss: 1.0927 - val_accuracy: 0.6799
Epoch 4/5
1050/1050 [==============================] - 300s 286ms/step - loss: 0.8035 - accuracy: 0.7549 - val_loss: 1.0556 - val_accuracy: 0.6914
Epoch 5/5
1050/1050 [==============================] - 297s 283ms/step - loss: 0.7006 - accuracy: 0.7854 - val_loss: 1.0773 - val_accuracy: 0.6958
